# SMCI/A: convergence with respect to surface resolution and computational times

In [ ]:
import os
import pandas as pd
from math import pi
import matplotlib.pyplot as plt
from matplotlib import rcParams
from itertools import cycle

from plot_study import *

rcParams["text.usetex"] = True
rcParams["figure.dpi"] = 300 
rcParams["font.size"] = 9
rcParams["axes.titlesize"] = 9
rcParams["axes.labelsize"] = 9
global_markers = ['o', 'x', '^', 'v', 'd']

In [ ]:
import warnings 
# Set the SMCIA_VOF_INIT_RESULTS environmental variable to the folder where you want the data to be saved. 
data_dir = os.curdir
try:
    data_dir = os.environ["SMCIA_VOF_INIT_RESULTS"] 
except:
    warnings.warn("No SMCIA_VOF_INIT_RESULTS environmental variable set, storing data in the current working directory.") 

### User input
Set the algorithm name (_SMCI_ or _SMCA_), the JSON file name and geometric parameters of the tested interfaces. The algorithm is necessary for an unambiguous naming of the saved plots.  
`group_by_params` determines how data is grouped to create subsets which are added to a single plot.

In [ ]:
algorithm = ''

# Allows the GitLab CI pipeline to select the algorithm
if not algorithm:
    algorithm = os.environ['VOF_INIT_ALGORITHM']

json_file_name = 'CPC2021-convergence-' + algorithm + '.json'
group_by_params = ['SURFACE', 'MESH_PERTURBATION']

# Sphere parameters
sphere_radius = 0.15 #  WARNING: Adapt this if another radius is chosen in templateCase/sphere.geo.template 

# Ellipsoid parameters
a = 0.4 # WARNING: Adapt this if another value is chosen in templateCase/ellipsoid.geo.template 
b = 0.3 # WARNING: Adapt this if another value is chosen in templateCase/ellipsoid.geo.template 
c = 0.2 # WARNING: Adapt this if another value is chosen in templateCase/ellipsoid.geo.template 

---
### Read data

In [ ]:
# Read dataframe and metadata
df = pd.read_json(json_file_name, orient='table')

# Try sorting index to avoid performance warnings according to
# https://stackoverflow.com/questions/54307300/what-causes-indexing-past-lexsort-depth-warning-in-pandas
df.sort_index(inplace=True)

### Compute analytic volumes and relative volume errors

In [ ]:
# Exact volume of the sphere.  
sphere_volume = sphere_radius**3 * 4 * pi / 3.

# Exact volume of the ellipsoid.  
ellipsoid_volume = 4 * a * b * c * pi / 3. 

# Compute volume errors with respect to exact volume
df['EXACT_VOLUME'] = sphere_volume
df.loc[(-1.0, 'triSurface', -1, 'ellipsoid', ), 'EXACT_VOLUME'] = ellipsoid_volume
df['VOLUME_ERROR_FROM_EXACT_VOLUME'] = ((df['VOLUME_FROM_VOLUME_FRACTION'] - df['EXACT_VOLUME'])/df['EXACT_VOLUME']).abs()

### Convergence with surface mesh resolution

In [ ]:
for params, subsetdf in df.groupby(level=group_by_params):
    fig, ax = plt.subplots()
    markers = cycle(global_markers)
    
    for n_cells, plotdf in subsetdf.groupby(level='N_CELLS_PER_DIM'):
        ax.plot(plotdf['N_TRIANGLES']**(1/2), plotdf['VOLUME_ERROR_FROM_EXACT_VOLUME'],
                label="$N_c = %s$" % n_cells, marker=next(markers))
        
    # Log scaling required for data:
    ax.loglog()
    
    # Show indicator for second order convergence
    add_convergence_order(ax, 2.0, xrelmin=0.22)
    
    # Set appropriate axes labels
    ax.set_ylabel("$E_v$")
    ax.set_xlabel("$\sqrt{N_T}$")
    
    # Show grid and legend
    ax.legend()
    ax.grid()
    
    # Generate file name
    plot_file_name = json_file_name.split('.')[0] + "-meshConvergence-" + algorithm + "-"
    for param, value in zip(group_by_params, params):
        plot_file_name = plot_file_name + param + str(value)
    plot_file_name = plot_file_name.replace('_', '-') + '.pdf'
    
    # Save PDF
    fig.savefig(os.path.join(data_dir, plot_file_name),
                    bbox_inches='tight')

    # Show the parameter values in the title:
    title = "Algorithm: " + algorithm + "\n"
    for param, value in zip(group_by_params, params):
        title = title + param.replace('_', ' ') + ": " + str(value) + "\n"
    ax.set_title(title)

### Computation times

In [ ]:
for params, subsetdf in df.groupby(level=group_by_params):
    fig, ax = plt.subplots()
    markers = cycle(global_markers)
    
    for n_cells, plotdf in subsetdf.groupby(level='N_CELLS_PER_DIM'):
        ax.plot(plotdf['N_TRIANGLES']**(1/2), plotdf['CPU_TIME_MICROSECONDS']/1e6,
                label="$N_c = %s$" % n_cells, marker=next(markers))
    
    # Set appropriate axes labels
    ax.set_ylabel("CPU time in seconds")
    ax.set_xlabel("$\sqrt{N_T}$")
    
    # Show grid and legend
    ax.legend()
    ax.grid()
    
    # Generate file name
    plot_file_name = json_file_name.split('.')[0] + "-CPUtimes-" + algorithm + "-"
    for param, value in zip(group_by_params, params):
        plot_file_name = plot_file_name + param + str(value)
    plot_file_name = plot_file_name.replace('_', '-') + '.pdf'
    
    # Save PDF
    fig.savefig(os.path.join(data_dir, plot_file_name),
                    bbox_inches='tight')

    # Show the parameter values in the title:
    title = "Algorithm: " + algorithm + "\n"
    for param, value in zip(group_by_params, params):
        title = title + param.replace('_', ' ') + ": " + str(value) + "\n"
    ax.set_title(title)